In [1]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [2]:
!pip install -q langchain-groq langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.7 MB/s eta 0:00:00


In [7]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [8]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [9]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [10]:
multiply.name

'multiply'

In [11]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [12]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [13]:
# tool binding

In [14]:
llm = ChatGroq(
    model_name="gemma2-9b-it",
)

In [15]:
llm.invoke('hi')

AIMessage(content='Hello! 👋 How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24, 'completion_time': 0.023636364, 'prompt_time': 0.001825698, 'queue_time': 0.066484562, 'total_time': 0.025462062}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--31044a62-4f9b-4410-8e8d-379107d3d494-0', usage_metadata={'input_tokens': 11, 'output_tokens': 13, 'total_tokens': 24})

In [16]:
llm_with_tools = llm.bind_tools([multiply])

In [17]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='<tool-use></tool-use> \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 968, 'total_tokens': 981, 'completion_time': 0.023636364, 'prompt_time': 0.04024658, 'queue_time': 0.067105155, 'total_time': 0.063882944}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b9bc0dca-2fd8-4e3d-984e-529740b37d0c-0', usage_metadata={'input_tokens': 968, 'output_tokens': 13, 'total_tokens': 981})

In [18]:
query = HumanMessage('can you multiply 3 with 1000')

In [19]:
messages = [query]

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [21]:
result = llm_with_tools.invoke(messages)

In [22]:
messages.append(result)

In [23]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tcrax5rbz', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 975, 'total_tokens': 1065, 'completion_time': 0.163636364, 'prompt_time': 0.065947778, 'queue_time': 0.11937322300000001, 'total_time': 0.229584142}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fbeb4482-e0d5-4c48-a806-1fac64178900-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'tcrax5rbz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 975, 'output_tokens': 90, 'total_tokens': 1065})]

In [24]:
tool_result = multiply.invoke(result.tool_calls[0])

In [25]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='tcrax5rbz')

In [26]:
messages.append(tool_result)

In [27]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tcrax5rbz', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 975, 'total_tokens': 1065, 'completion_time': 0.163636364, 'prompt_time': 0.065947778, 'queue_time': 0.11937322300000001, 'total_time': 0.229584142}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fbeb4482-e0d5-4c48-a806-1fac64178900-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'tcrax5rbz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 975, 'output_tokens': 90, 'total_tokens': 1065}),
 ToolMessage(content='3000', name='multiply', tool_call_id='tcrax5rbz')]

In [28]:
llm_with_tools.invoke(messages).content

'3000'

In [29]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [30]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [31]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1750291201,
 'time_last_update_utc': 'Thu, 19 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1750377601,
 'time_next_update_utc': 'Fri, 20 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 86.4882}

In [32]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [33]:
# tool binding
llm = ChatGroq(model_name = 'gemma2-9b-it')

In [34]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [35]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [36]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [49]:
ai_message = llm_with_tools.invoke(messages)

In [50]:
messages.append(ai_message)

In [51]:
ai_message.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '0mvbt8mbs',
  'type': 'tool_call'}]

In [52]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [53]:
tool_message2

ToolMessage(content='0.11560000000000001', name='convert', tool_call_id='0mvbt8mbs')

In [46]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mnztxzeqd', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1399, 'total_tokens': 1452, 'completion_time': 0.096363636, 'prompt_time': 0.046190356, 'queue_time': 0.093159125, 'total_time': 0.142553992}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d0e6d6dd-31dc-41b5-9590-44e2b4ac1dba-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10}, 'id': 'mnztxzeqd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1399, 'output_tokens': 53, 'total_tokens': 1452})

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to help! How can I assist you today?

> Finished chain.
